<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
evts = load_sample_events()
exs = [first(evts, risinstance(o)) for o in described_evts]

## Animated Stats

This section outlines how we can display statistics and visualizations such as sparklines and status bars that are animated as events are received.

In [1]:
#|output: asis
#| echo: false
show_doc(EProg)

---

[source](https://github.com/fastai/ghtop/tree/master/blob/master/ghtop/richext.py#L18){target="_blank" style="float:right; font-size:smaller"}

### EProg

>      EProg (hdg='Quota', width=10)

Progress bar with a heading `hdg`.

When you instantiate `Eprog` the starting progress is set to 0%:

In [ ]:
p = EProg()
console.print(p)

Output()

You can update the progress bar with the `update` method:

In [ ]:
p.update(10)
console.print(p)

### `Espark` - A sparkline combined with an EventTimer

fastcore's `EventTimer` calculates frequency metrics aggregated by slices of time specified by the argument `span`.  The `EventTimer` can produce a sparkline that shows the last n time slices, where n is specified by the parameter `store`:

In [2]:
#|output: asis
#| echo: false
show_doc(ESpark)

---

[source](https://github.com/fastai/ghtop/tree/master/blob/master/ghtop/richext.py#L30){target="_blank" style="float:right; font-size:smaller"}

### ESpark

>      ESpark (nm:str, color:str, ghevts=None, store=5, span=0.2, mn=0, mx=None,
>              stacked=True, show_freq=False)

An `EventTimer` that displays a sparkline with a heading `nm`.

In [ ]:
from time import sleep

In [ ]:
def _randwait(): yield from (sleep(random.random()/200) for _ in range(100))

c = EventTimer(store=5, span=0.03)
for o in _randwait(): c.add(1)

By default `nm` will be stacked on top of the sparkline.  We simulate adding events to [`ESpark`](https://ghtop.fast.ai/richext.html#espark) and render the result:

In [ ]:
e = ESpark(nm='💌Issue', color='blue', store=5)

def _r(): return random.randint(1,30)

def _sim(e, steps=8, sleep=.2):
    for i in range(steps):
        e.add(_r())
        time.sleep(sleep)

_sim(e)
console.print(e)

If you would prefer `nm` and the sparkline to be on one line instead, you can set `stacked` to `false`:

In [ ]:
e = ESpark(color='blue', nm='💌Issue', stacked=False)
_sim(e)
console.print(e)

You can optionally specify a list of `GhEvent` types that will allow you to update sparklines by streaming in events. `described_evts` has a complete list of options:

In [ ]:
described_evts

(ghapi.event.PushEvent,
 ghapi.event.CreateEvent,
 ghapi.event.IssueCommentEvent,
 ghapi.event.WatchEvent,
 ghapi.event.PullRequestEvent,
 ghapi.event.PullRequestReviewEvent,
 ghapi.event.PullRequestReviewCommentEvent,
 ghapi.event.DeleteEvent,
 ghapi.event.ForkEvent,
 ghapi.event.IssuesEvent,
 ghapi.event.ReleaseEvent,
 ghapi.event.MemberEvent,
 ghapi.event.CommitCommentEvent,
 ghapi.event.GollumEvent,
 ghapi.event.PublicEvent)

If `ghevts` is specified, only events that match the list of the `GhEvent` types will increment the event counter. 

In the below example, the `IssueCommentEvent` and `IssuesEvent` are listed, therefore any other event types will not update the event counter:

In [ ]:
_pr_evts = evts.filter(risinstance((PullRequestEvent, PullRequestReviewCommentEvent, PullRequestReviewEvent)))
_watch_evts = evts.filter(risinstance((WatchEvent)))


_s = ESpark('Issues', 'blue', [IssueCommentEvent, IssuesEvent], span=5)
_s.add_events(_pr_evts)
_s.add_events(_watch_evts)
test_eq(_s.events, 0)

However, events that match those types will update the event counter accordingly:

In [ ]:
_issue_evts = evts.filter(risinstance((IssueCommentEvent, IssuesEvent)))
_s.add_events(_issue_evts)
test_eq(_s.events, len(_issue_evts))

If `ghevts` is not specified, all events are counted:

In [ ]:
_s = ESpark('Issues', 'blue', span=5)
_s.add_events(evts)
test_eq(_s.events, len(evts))

You can also just add one event at a time instead of a list of events:

In [ ]:
_s = ESpark('Issues', 'blue', span=5)
_s.add_events(evts[0])
test_eq(_s.events, 1)

## Update A Group of Sparklines with [`SpkMap`](https://ghtop.fast.ai/richext.html#spkmap)

In [3]:
#|output: asis
#| echo: false
show_doc(SpkMap)

---

[source](https://github.com/fastai/ghtop/tree/master/blob/master/ghtop/richext.py#L57){target="_blank" style="float:right; font-size:smaller"}

### SpkMap

>      SpkMap (spks:List[__main__.ESpark])

A Group of `ESpark` instances.

You can define a [`SpkMap`](https://ghtop.fast.ai/richext.html#spkmap) instance with a list of [`ESpark`](https://ghtop.fast.ai/richext.html#espark):

In [ ]:
s1 = ESpark('Issues', 'green', [IssueCommentEvent, IssuesEvent], span=60)
s2 = ESpark('PR', 'red', [PullRequestEvent, PullRequestReviewCommentEvent, PullRequestReviewEvent], span=60)
s3 = ESpark('Follow', 'blue', [WatchEvent, StarEvent, IssueCommentEvent, IssuesEvent], span=60)
s4 = ESpark('Other', 'red', span=60)

sm = SpkMap([s1,s2,s3,s4])

We haven't added any events to [`SpkMap`](https://ghtop.fast.ai/richext.html#spkmap) so the event count will be zero for all sparklines:

In [ ]:
sm.evcounts

{'Issues': 0, 'PR': 0, 'Follow': 0, 'Other': 0}

In the above example, Issue events update both the `Issues` and `Follow` sparklines, as well as the `Other` sparkline which doesn't have any `GhEvent` type filters so it counts all events:

In [ ]:
sm.add_events(_issue_evts)
test_eq(sm.evcounts['Issues'], len(_issue_evts))
test_eq(sm.evcounts['Follow'], len(_issue_evts))
test_eq(sm.evcounts['Other'], len(_issue_evts))

sm.evcounts

{'Issues': 80, 'PR': 0, 'Follow': 80, 'Other': 80}

You can also just add one event at a time:

In [ ]:
sm.add_events(_pr_evts[0])
test_eq(sm.evcounts['PR'], 1)
test_eq(sm.evcounts['Other'], len(_issue_evts)+1)

It may be desirable to make certain attributes of the sparklines the same so the group can look consistent.  For example, by default sparklines are set to `stacked=True`, which means the labels are on top:

In [ ]:
console.print(sm)

We can update `stack=False` for the entire group with the `update_params` method:

In [ ]:
sm.update_params(stacked=False)
console.print(sm)

In [ ]:
sm.update_params(stacked=True, span=.1, store=8)
def _sim(s):
    with Live(s) as live:
        for i in range(200):
            s.add_events(evts[:random.randint(0,500)])
            time.sleep(random.randint(0,10)/100)
_sim(sm)

Output()

In [ ]:
console.print(sm.spks[0])

### Stats - Sparklines, Progress bars and Counts Combined

We may want to combine sparklines (with [`ESpark`](https://ghtop.fast.ai/richext.html#espark)), spinners, and progress bars (with [`EProg`](https://ghtop.fast.ai/richext.html#eprog)) to display organized information concerning an event stream.  [`Stats`](https://ghtop.fast.ai/richext.html#stats) helps you create, group, display and update these elements together.

In [4]:
#|output: asis
#| echo: false
show_doc(Stats)

---

[source](https://github.com/fastai/ghtop/tree/master/blob/master/ghtop/richext.py#L80){target="_blank" style="float:right; font-size:smaller"}

### Stats

>      Stats (spks:List[__main__.ESpark], store=None, span=None, stacked=None,
>             show_freq=None, max_width=75, spin:str='earth', spn_lbl='/min')

Renders a group of `ESpark` along with a spinner and progress bar that are dynamically sized.

Instantiate [`Stats`](https://ghtop.fast.ai/richext.html#stats) with a list of `Espark` instances.  The parameters: `store`, `span`, and `stacked` allow you to set or override properties of underlying sparklines for consistency.  

In [ ]:
s1 = ESpark('Issues', 'green', [IssueCommentEvent, IssuesEvent])
s2 = ESpark('PR', 'red', [PullRequestEvent, PullRequestReviewCommentEvent, PullRequestReviewEvent])
s3 = ESpark('Follow', 'blue', [WatchEvent, StarEvent])
s4 = ESpark('Other', 'red')

s = Stats([s1,s2,s3,s4], store=5, span=.1, stacked=True)
console.print(s)

Output()

You can add events to update counters and sparklines just like [`SpkMap`](https://ghtop.fast.ai/richext.html#spkmap):

In [ ]:
s.add_events(evts)
console.print(s)

You can update the progress bar with the `update_prog` method: 

In [ ]:
s.update_prog(50)
console.print(s)

Here is what this looks like when animated using `Live`:

In [ ]:
def _sim_spark(s):
    with Live(s) as live:
        for i in range(101):
            s.update_prog(i)
            s.add_events(evts[:random.randint(0,500)])
            time.sleep(random.randint(0,10)/100)

s.update_params(span=1, show_freq=True)
_sim_spark(s)

Output()

## Event Panel

Display GitHub events in a `FixedPanel`, which is a frame of fixed height that displays streaming data.

In [ ]:
p = FixedPanel(15, box=box.HORIZONTALS, title='ghtop')
for e in evts[:163]: p.append(e)
p

#### Using `grid` with `FixedPanel`

We can use `grid` to arrange multiple `FixedPanel` instances in rows and columns.  Below is an example of how two `FixedPanel` instances can be arranged in a row:

In [ ]:
p = FixedPanel(15, box=box.HORIZONTALS, title='ghtop')
for e in exs: p.append(e)
grid([[p,p]])

Here is another example of a four `FixedPanel` instances arranged in two rows and two columns:

In [ ]:
types = IssueCommentEvent,IssuesEvent,PullRequestEvent,PullRequestReviewEvent
ps = {o:FixedPanel(15, box=box.HORIZONTALS, title=camel2words(remove_suffix(o.__name__,'Event'))) for o in types}

In [ ]:
for k,v in ps.items(): v.extend(evts.filter(risinstance(k)))
isc,iss,prs,prrs = ps.values()
grid([[isc,iss],[prs,prrs]], width=110)

In [ ]:
#| include: false
from nbdev import nbdev_export
nbdev_export()

Converted 00_ghtop.ipynb.
Converted index.ipynb.
Converted richext.ipynb.
